Log in to LinkedIn?



In [ ]:
var notRobot = () => {
    return client.click('')
}

var enterLinkedIn = () => {
    console.log('LinkedIn: Sign in required');
    var credentials = getCredentials('linkedin.com');
    return client.click('input[name="session_key"]')
        .keys(credentials.session_key)
        .pause(1000)
        .then(() => console.log('LinkedIn: Require password'))
        .click('input[name="session_password"]')
        .keys(credentials.session_password)
        .submitForm('.login-form');
}

var loginLinkedIn = () => {
    return client.url('https://www.linkedin.com/')
        .pause(1000)
        .isExisting('*=Forgot password?').then(is => is 
            ? enterLinkedIn()
            : Promise.resolve([]));
};
module.exports = loginLinkedIn;
loginLinkedIn;


Scrape my own LinkedIn profile?



In [ ]:
var visitMyProfile = () => {
    // TODO: add check for needing to go to LinkedIn
    // TODO: add check for needing to login
    return client.click('#nav-settings__dropdown-trigger')
        .pause(500)
        .click('.nav-settings__view-profile-link')
        .pause(2000);
}

var readLinkedInProfileInfo = () => {
    var name, title, url, phone, email;
    return client
        .getUrl().then(url => url.indexOf('/in/') == -1
            ? visitMyProfile()
            : Promise.resolve([]))
        .click('.contact-see-more-less')
        .then(() => Promise.all([
            client.getText('.pv-top-card-section__name'),
            client.getText('.pv-top-card-section__headline'),
            client.getText('.ci-vanity-url .pv-contact-info__contact-item'),
            // is existing?
            client.getText('.ci-phone .pv-contact-info__contact-item'),
            client.getText('.ci-email .pv-contact-info__contact-item')
        ]))
        .then(r => ({
            name: r[0],
            title: r[1],
            url: r[2],
            phone: r[3],
            email: r[4],
        }));
};
module.exports = readLinkedInProfileInfo;
readLinkedInProfileInfo;


List all LinkedIn threads?



In [ ]:
var importer = require('../Core');

var scrollLinkedInThreads = (threads) => {
    // TODO: add check for needing to go to LinkedIn
    // TODO: add check for needing to login
    // scroll to bottom of messages
    return client
        .execute(() => {
            document.getElementsByClassName(
                'msg-conversations-container__conversations-list')[0]
                .scrollTop += 10000;
        })
        .pause(2000)
        .then(() => getLinkedInThreads(threads));
}

var getLinkedInThreads = (threads) => {
    return client    
        .execute(() => {
            var iterator = document.evaluate(
                '//*[@data-control-name="view_message"]/@href',
                document, null,
                XPathResult.UNORDERED_NODE_ITERATOR_TYPE, null);
            var co = []; var m;
            while ((m = iterator.iterateNext()) && co.push(m.value));
            return co;
        })
        .then(els => {
            var newThreads = els.value
                .filter(e => threads.indexOf(e) === -1);
            threads = threads.concat(newThreads);
            return newThreads.length > 0
                ? scrollLinkedInThreads(threads)
                : Promise.resolve(threads);
        });
}

var listLinkedInThreads = (threads) => {
    return client.getUrl().then(url => url.indexOf('/messaging') == -1
            ? client.click('a[href*="/messaging/"]')
            : Promise.resolve([]))
        .pause(2000)
        .then(() => getLinkedInThreads(threads || []))
};
module.exports = listLinkedInThreads;
listLinkedInThreads;


Read all messages in LinkedIn thread?



In [18]:
var importer = require('../Core');
var moment = require('moment');
var chrono = require('chrono-node');

var getThreadParticipants = (thread) => {
    var profiles = []
    // check for thread url and go there
    return client.getUrl().then(url => url.indexOf(thread + 'topcard/') == -1
        ? client.url('https://www.linkedin.com' + thread + 'topcard/')
        : Promise.resolve([]))
    .pause(3000)
    .elements('.view-profile')
    .then(els => {
        return Promise.all(els.value.map(a => 
            client.elementIdAttribute(a.ELEMENT, 'href')));
    })
    .then(els => {
        profiles = els.map(el => el.value);
        return importer.interpretAll(['scrape LinkedIn profile']);
    })
    .then(r => r[0].runFresherInNewContext({client}))
    .then(readLinkedInProfileInfo => {
        return importer.runAllPromises(profiles
            .map(p => client.url(p)
                 .then(() => readLinkedInProfileInfo())))
    });
};

var scrollLinkedInMessages = (messages) => {
    // TODO: add check for needing to go to LinkedIn
    // TODO: add check for needing to login
    // scroll to bottom of messages
    return client
        .execute(() => {
            document.getElementsByClassName(
                'msg-thread')[0]
                .scrollTop -= 10000;
        })
        .pause(2000)
        .then(() => readLinkedInMessages(messages));
}

var readLinkedInMessages = (messages) => {
    // TODO: check for thread id in url?
    return client.execute(() => {
        var groups = document.getElementsByClassName(
            'msg-s-message-list__date-group');
        var result = [];
        for(var i = 0; i < groups.length; i++) {
            var times = groups[i].getElementsByTagName('time');
            var messages = groups[i].getElementsByClassName(
                'msg-s-message-group');
            for(var j = 0; j < messages.length; j++) {
                var profile = messages[j].getElementsByTagName('img')[0];
                var from;
                if(typeof profile !== 'undefined') {
                    from = profile.getAttribute('title');
                } else {
                    from = 'Me';
                }
                var bubbles = messages[j].getElementsByClassName(
                    'msg-s-message-listitem__message-bubble')
                for(var k = 0; k < bubbles.length; k++) {
                    result[result.length] = {
                        from: from,
                        time: times[0].innerText + ' '  + times[j+1].innerText,
                        message: bubbles[k].innerText
                    };
                }
            }
        }
        return result;
    }).then(r => {
        var newMessages = r.value
            .filter(e => messages
                .filter(m => m.message == e.message).length === 0);
        messages = newMessages.concat(messages);
        return newMessages.length > 0
                ? scrollLinkedInMessages(messages)
                : Promise.resolve(messages);
    });
};

var readLinkedInThread = (thread, messages) => {
    var participants;
    return client.getUrl().then(url => url.indexOf(thread) == -1
        ? client.url('https://www.linkedin.com' + thread)
        : Promise.resolve([]))
    .then(() => getThreadParticipants(thread))
    .then(p => participants = p)
    // TODO: save to contacts
    .url('https://www.linkedin.com' + thread)
    .then(() => readLinkedInMessages(messages || []))
    .then(messages => ({
        thread: thread,
        participants: participants,
        messages: messages.map(m => {
            var newTime = chrono.parseDate(m.time);
            newTime.setHours(newTime.getHours()-(new Date()).getTimezoneOffset()/60)
            return Object.assign(m, {time: newTime})
        })
    }));
};
module.exports = readLinkedInThread;
readLinkedInThread;



[Function: readLinkedInThread]

In [8]:
var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var fs = require('fs');
var project = PROFILE_PATH + '/Conversations';
var importer = require('../Core');
var glob = require('glob');

$$.async()
var runSeleniumCell;
var threads, readLinkedInThread;

var readThread = (thread) => {
    var threadId = thread.replace(/^\/|\/$/ig, '').split('/').pop();
    var file = glob.sync('**/' + threadId + '-*.json', {cwd: project})[0];
    var messages;
    try { messages = require(file) }
    catch (e) { messages = [] }
    return readLinkedInThread(thread, messages)
    .then(t => {
        var filename = project + '/' 
            + t.thread.replace(/^\/|\/$/ig, '').split('/').pop()
            + '-' + t.participants
            .map(p => p.name).join('')
            .replace(/^[a-z]/ig, '_') + '.json';
        fs.writeFileSync(filename, JSON.stringify(t, null, 4));
        return t;
    });
};

var listThreads = () => {
    return runSeleniumCell('log in linkedin')
    .then(loginLinkedIn => loginLinkedIn())
    .then(() => runSeleniumCell('scrape LinkedIn profile'))
    .then(readLinkedInProfileInfo => readLinkedInProfileInfo())
    // show my profile info
    .then(() => runSeleniumCell('list all LinkedIn threads'))
    .then(listLinkedInThreads => {
        var threads;
        try { threads = require(project); } 
        catch (e) { threads = []; }
        return listLinkedInThreads(threads);
    })
    // list all message threads
    .then(r => fs.writeFileSync(
        project + '/threads.json',
        JSON.stringify(r, null, 4)));
};

importer.interpretAll(['selenium cell'])
    .then(r => {
        runSeleniumCell = r[0].runInNewContext();
    //    return listThreads();
    })
    // scrape each thread
    .then(() => {
        try { threads = require(project + '/threads.json'); }
        catch (e) { threads = []; }
        return runSeleniumCell('messages in LinkedIn thread');
    })
    .then(r => {
        readLinkedInThread = r;
        return readThread(threads[0]);
    /*
        return Promise.all(.slice(0, 10).map(t => 
            client.newWindow()
            .then(() => readThread(threads[0]))
            .end()));
            */
    })
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e));



Already imported
[ 'import notebook.ipynb', 'interpret all notebooks.ipynb' ]
Already imported
[ 'import notebook.ipynb', 'interpret all notebooks.ipynb' ]
Initializing webdriver on localhost
Already imported
[ 'import notebook.ipynb', 'interpret all notebooks.ipynb' ]
Already imported
[ 'import notebook.ipynb', 'interpret all notebooks.ipynb' ]


{ thread: '/messaging/thread/6301495491385442304/',
  participants: 
   [ { name: 'Sam Shammah',
       title: 'Founder and CEO of Akwai.com',
       url: 'linkedin.com/in/akwai',
       phone: '480-382-5023 (Work)',
       email: 'sam@twowks.com' } ],
  messages: 
   [ { from: 'Sam Shammah',
       time: 2017-08-17T19:33:00.000Z,
       message: 'Hey Brian. How is MindSpree coming along? You probably didn\'t know this but to keep it lean, I\'ve been doing freelance work in addition to growing Akwai. I\'m looking to get a few more projects in before school starts and our busy season kicks in. Take a look at some of my previous work if you think an explainer video could be beneficial for any of your endeavors www.akwai.com/video\n' },
     { from: 'Me',
       time: 2017-08-17T19:35:00.000Z,
       message: 'So you are looking for work?\n' },
     { from: 'Sam Shammah',
       time: 2017-08-17T19:36:00.000Z,
       message: 'Yea I want to get a couple more projects in this month\n' },
 